# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests



# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ilulissat,69.2167,-51.1000,32.02,93,100,17.27,GL,1697224348
1,1,edinburgh of the seven seas,-37.0676,-12.3116,59.31,95,100,29.10,SH,1697224332
2,2,port mathurin,-19.6833,63.4167,72.66,72,24,12.48,MU,1697224330
3,3,almeirim,-1.5233,-52.5817,98.85,32,82,6.06,BR,1697224299
4,4,luwuk,-0.9516,122.7875,75.54,79,39,4.25,ID,1697224554


---
### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
%%capture --no-display
# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)

map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM", 
    width=800,
    height=600,
    size="Humidity",
    color="City",
    alpha=0.75  
)

# Display the map
map_plot1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp_condition = (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)
wind_speed_condition = city_data_df["Wind Speed"] < 4.5
cloudiness_condition = city_data_df["Cloudiness"] == 0

filtered_cities_df = city_data_df[max_temp_condition & wind_speed_condition & cloudiness_condition]
# Drop any rows with null values
filtered_cities_df=city_data_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ilulissat,69.2167,-51.1000,32.02,93,100,17.27,GL,1697224348
1,1,edinburgh of the seven seas,-37.0676,-12.3116,59.31,95,100,29.10,SH,1697224332
2,2,port mathurin,-19.6833,63.4167,72.66,72,24,12.48,MU,1697224330
3,3,almeirim,-1.5233,-52.5817,98.85,32,82,6.06,BR,1697224299
4,4,luwuk,-0.9516,122.7875,75.54,79,39,4.25,ID,1697224554


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,ilulissat,69.2167,-51.1000,32.02,93,100,17.27,GL,1697224348,
1,1,edinburgh of the seven seas,-37.0676,-12.3116,59.31,95,100,29.10,SH,1697224332,
2,2,port mathurin,-19.6833,63.4167,72.66,72,24,12.48,MU,1697224330,
3,3,almeirim,-1.5233,-52.5817,98.85,32,82,6.06,BR,1697224299,
4,4,luwuk,-0.9516,122.7875,75.54,79,39,4.25,ID,1697224554,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "sort":"distance",
    "type":"hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius},{latitude},{longitude}"
    params["bias"] = f"point:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ilulissat - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
almeirim - nearest hotel: No hotel found
luwuk - nearest hotel: No hotel found
saudarkrokur - nearest hotel: No hotel found
polyarnyy - nearest hotel: No hotel found
ponta delgada - nearest hotel: No hotel found
enewetak - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
bandarbeyla - nearest hotel: No hotel found
sandnessjoen - nearest hotel: No hotel found
sansanne-mango - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
margate - nearest hotel: No hotel found
darzab - nearest hotel: No hotel found
kapuskasing - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
weyburn - nearest hotel: No hotel found
puerto natales - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
soure - nearest hotel: No hotel found
fukuecho - neares

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,ilulissat,69.2167,-51.1000,32.02,93,100,17.27,GL,1697224348,No hotel found
1,1,edinburgh of the seven seas,-37.0676,-12.3116,59.31,95,100,29.10,SH,1697224332,No hotel found
2,2,port mathurin,-19.6833,63.4167,72.66,72,24,12.48,MU,1697224330,No hotel found
3,3,almeirim,-1.5233,-52.5817,98.85,32,82,6.06,BR,1697224299,No hotel found
4,4,luwuk,-0.9516,122.7875,75.54,79,39,4.25,ID,1697224554,No hotel found
5,5,saudarkrokur,65.7461,-19.6394,29.35,86,100,14.20,IS,1697224555,No hotel found
6,6,polyarnyy,69.1989,33.4478,34.57,79,18,6.40,RU,1697224375,No hotel found
7,7,ponta delgada,37.7333,-25.6667,72.32,78,40,18.41,PT,1697224555,No hotel found
8,8,enewetak,11.3474,162.3373,82.44,75,72,5.53,MH,1697224361,No hotel found
9,9,port-aux-francais,-49.3500,70.2167,36.21,96,87,23.24,TF,1697224311,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    x="Lng",
    y="Lat",
    geo=True,
    size=100,
    alpha=0.7,
    tiles="CartoLight",
    hover_cols=["City", "Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)